In [6]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import yfinance as yf

In [7]:
train_df = pd.read_csv("data_train.csv")


In [8]:
variable_names = ['B','Tc', 'beta', 'C', 'Omega', 'Phi', 'RMSE']

means = train_df[variable_names].mean()
stds = train_df[variable_names].std()

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.ModuleList()
        
        # Add input layer
        self.layers.append(nn.Linear(7, 128))
        
         # Add input layer
        self.layers.append(nn.Linear(128, 64))
        
        # Add input layer
        self.layers.append(nn.Linear(64, 32))
        
        # Add output layer
        self.layers.append(nn.Linear(32, 1))
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.relu(layer(x))
        x = self.sigmoid(self.layers[-1](x))
        return x

In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("best_model_240427.pth")) #test -> docker container
model.eval()

NeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=7, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [12]:
Mean = means.values.tolist()

In [13]:
SD = stds.values.tolist()

In [14]:
## 데이터

In [15]:
Data_folder = "LPPL"

In [16]:
file_names = os.listdir(Data_folder)

In [30]:
Result_list = []

In [31]:
for file_name in file_names:
    
    if file_name == '.DS_Store':
        continue
    
    ticker, date_raw = file_name.split("_")[:3] 
    date = date_raw[:-5]
    
    # CSV 파일 로드
    df = pd.read_excel(Data_folder + "/" + file_name)

    # "RMSE"가 가장 작은 행 찾기
    min_rmse_row = df.loc[df['RMSE'].idxmin()]

    # "A", "B", "Tc", "beta", "C", "Omega", "Phi" 값 추출
    A_value = min_rmse_row['A']
    B_value = min_rmse_row['B']
    Tc_value = min_rmse_row['Tc']
    beta_value = min_rmse_row['beta']
    C_value = min_rmse_row['C']
    Omega_value = min_rmse_row['Omega']
    Phi_value = min_rmse_row['Phi']
    RMSE_value = min_rmse_row['RMSE']
    
    PD = [B_value,Tc_value, beta_value, C_value, Omega_value, Phi_value, RMSE_value]
    
    for i in range(0,5):
        PD[i] = ((PD[i]-Mean[i])/SD[i])
        
    preprocessed_data = np.array([PD])  # Replace with actual preprocessed data
    input_tensor = torch.tensor(preprocessed_data, dtype=torch.float32)  # Convert to tensor
    
    # Make predictions
    with torch.no_grad():
        predictions = model(input_tensor)
        
    Result_list.append({"Ticker": ticker, 'date': date, 'AI': predictions.item(), 'tc': Tc_value})

In [32]:
Result = pd.DataFrame(Result_list)

In [33]:
Result.to_csv("result_AI.csv", index=False, encoding='utf-8-sig')